In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

from catboost import Pool, CatBoostClassifier
from sklearn.metrics import roc_curve, auc, precision_score, confusion_matrix,\
 ConfusionMatrixDisplay, recall_score, accuracy_score, f1_score, classification_report, make_scorer
import pickle
import matplotlib.pyplot as plt

In [2]:
# Preprocess data
def process_data(name):
    file = r'../data/' + name + '.csv'
    print("processing file: {}".format(file))
    file = pd.read_csv(r'../data/cases_train_preprocessed.csv',dtype=object)
    file.drop(columns=['land','Last_Update','province','country'],inplace=True)
    file['source'] = file['source'].fillna(value='')
    return file
file = process_data("cases_train_preprocessed")

processing file: ../data/cases_train_preprocessed.csv


In [3]:
# Train / Test split for model evaluation
def catboost_data(file):
    cat_feature=['sex','source','age_range_ind','age_range']
    text_feature = ['additional_information']

    df_tr, df_va = train_test_split(file, test_size=0.2, random_state = 1)

    df_tr_x = df_tr.drop(columns="outcome")
    df_tr_y = df_tr['outcome']

    df_tr_data = Pool(data = df_tr_x,
                   label = df_tr_y,
                   cat_features = cat_feature, 
                   text_features=text_feature)

    df_va_x = df_va.drop(columns="outcome")
    df_va_y = df_va['outcome']

    df_va_data = Pool(data = df_va_x,
                   label = df_va_y,
                   cat_features = cat_feature, 
                   text_features=text_feature)
    return df_tr_x, df_tr_y, df_va_x, df_va_y
tr, tr_y, va, va_y = catboost_data(file)

In [ ]:
# Specifying possible parameters of catboost
grid = {'learning_rate': [0.1, 0.2, 0.3],
        'depth': [6, 8, 10, 14, 10],
        'l2_leaf_reg': [0.2, 0.4, 0.6],
        'n_estimators':[400, 500, 600]
       }

# Specify catboost parameters
cat_feature=['sex','source','age_range_ind','age_range']
text_feature = ['additional_information']

# Creating required scoring function
scorer = {
    'f1_macro' : make_scorer(f1_score, average='macro'),
    'recall_macro': make_scorer(recall_score , average='macro'),
    'accuracy': make_scorer(accuracy_score),
    'recall_d' : make_scorer(recall_score,average=None,labels=['deceased']),
    'f1_d' : make_scorer(f1_score, average=None, labels=['deceased']),
}

# Creating Catboost model
model = CatBoostClassifier( task_type="GPU",
                            devices='0:1',
                            loss_function='MultiClass',
                            auto_class_weights = 'SqrtBalanced',
                            cat_features = cat_feature,
                            text_features = text_feature)

clf_grid = GridSearchCV(estimator=model, param_grid=grid, scoring=scorer, cv= StratifiedKFold(n_splits=5), refit="f1_d", n_jobs=-1)
clf = clf_grid.fit(tr, tr_y)

In [ ]:
print(clf_grid.best_params_)
df = pd.DataFrame(clf_grid.cv_results_ )
df = df.sort_values(by='rank_test_f1_d')
df = df[df.columns.drop(list(df.filter(regex=r'(time)|(std)|(split)|(param_)|(rank)')))]
df = df[['params', 'mean_test_f1_d', 'mean_test_recall_d', 'mean_test_accuracy', 'mean_test_recall_macro','mean_test_f1_macro']]
df.rename(columns={"mean_test_f1_macro": "Overall F1-Score(Macro)", 
                   "mean_test_recall_macro": "Overall Recall(Macro)",
                   "mean_test_accuracy": "Overall Accuracy",
                   "mean_test_f1_d": "F1-Score on Deceased",
                   "mean_test_recall_d": "Recall on Deceased",
                   "params": "Hyperparameters"
                  },inplace=True)
df.to_csv('../results/catboost_tuning.csv',index=False)
df

In [ ]:
label = ['hospitalized', 'nonhospitalized','recovered', 'deceased']
va_pred = clf.predict(va)
ev_va_report = classification_report(va_y, va_pred)
ev_va_accuracy = accuracy_score(va_y, va_pred)
ev_va_matrix = confusion_matrix(va_y, va_pred,labels=label)
print(ev_va_report)
ConfusionMatrixDisplay(ev_va_matrix).plot()

In [ ]:
# # Check if prediction file is valid
# def check_if_file_valid(filename):
#     assert filename.endswith('../results/predictions.txt'), 'Incorrect filename'
#     f = open(filename).read()
#     l = f.split('\n')
#     assert len(l) == 46500, 'Incorrect number of items'
#     assert (len(set(l)) == 4), 'Wrong class labels'
#     return 'The predictions file is valid'
# check_if_file_valid('predictions.txt')